In [1]:
using LinearAlgebra
using Distributed
using SharedArrays
using Plots
using DataFrames, CSV

In [2]:
using BeyondHulten

## Loading in Data

In [6]:
data = read_data_cb("I-O_DE2019_formatiert.csv")

CBData(84×87 DataFrame
 Row │ Sektoren                           Erz. d. Landwirtschaft, Jagd und Die ⋯
     │ String                             Int64                                ⋯
─────┼──────────────────────────────────────────────────────────────────────────
   1 │ Erz. d. Landwirtschaft, Jagd und…                                       ⋯
   2 │ Forstwirtschaftl. Erzeugnisse un…
   3 │ Fische, Fischerei- und Aquakultu…
   4 │ Kohle
   5 │ Erdöl und Erdgas                                                        ⋯
   6 │ Erze, Steine und Erden, sonstige…
   7 │ Nahrungs- u. Futtermittel, Geträ…
   8 │ Textilien, Bekleidung, Leder und…
  ⋮  │                 ⋮                                          ⋮            ⋱
  78 │ Bruttolöhne und -gehälter                                               ⋯
  79 │ Sonst.Produktionsabgaben abzgl. …
  80 │ Abschreibungen
  81 │ Nettobetriebsüberschuss
  82 │ Bruttowertschöpfung                                                     ⋯
  83 │ Produktion

In [22]:
elasticities = CBELasticities(data.factor_share,1 .- data.factor_share)

CBELasticities([0.33869040952700935, 0.5536193029490617, 0.37474541751527496, 0.4088888888888889, 0.5505084745762712, 0.33076923076923076, 0.21807865892972275, 0.2916315205327414, 0.2540205538558092, 0.23447542319181677  …  0.627565017008669, 0.6781704551739395, 0.7656068719321731, 0.6808288921308839, 0.651309137981537, 0.6442910187978649, 0.5929179709852234, 0.7169902465955098, 0.6560055223193741, 0.6805914494572842], [0.6613095904729906, 0.4463806970509383, 0.6252545824847251, 0.5911111111111111, 0.44949152542372883, 0.6692307692307693, 0.7819213410702772, 0.7083684794672587, 0.7459794461441909, 0.7655245768081832  …  0.37243498299133104, 0.3218295448260605, 0.23439312806782686, 0.31917110786911607, 0.348690862018463, 0.3557089812021351, 0.40708202901477664, 0.2830097534044902, 0.34399447768062585, 0.3194085505427158])

In [24]:
demand_shock = ones(71)
supply_shock = ones(71)
shocks = Shocks(supply_shock, demand_shock)
#shocks.supply_shock[1:3] .= 1.01
#sector = ["Vorb.Baustellen-,Bauinstallations-,Ausbauarbeiten"]
#investment = [25000]
#calculate_investment!(shocks, data, investment, sector)

Shocks([1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0  …  1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0], [1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0  …  1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0])

In [25]:
solution = solve_cobb_douglas_modell(data, shocks, elasticities)

Row,prices,quantities,sectors,value_added
,Float64,Float64,String,Float64
1,1.59731e-31,0.0168675,"Erz. d. Landwirtschaft, Jagd und Dienstleistungen",9.1252e-34
2,1.41768e-31,0.00214754,Forstwirtschaftl. Erzeugnisse und Dienstleistungen,1.6855e-34
3,1.55938e-31,0.000154877,"Fische, Fischerei- und Aquakulturerzeugnisse, DL",9.05053e-36
4,1.15335e-31,0.000962584,Kohle,4.53949e-35
5,8.99622e-32,0.000544615,Erdöl und Erdgas,2.6972e-35
6,1.56587e-31,0.00395925,"Erze, Steine und Erden, sonstige Bergbauerz., DL",2.05065e-34
7,1.58811e-31,0.0710957,"Nahrungs- u. Futtermittel, Getränke, Tabakerzeugn.",2.46228e-33
8,1.47195e-31,0.00836887,"Textilien, Bekleidung, Leder und Lederwaren",3.59249e-34
9,1.3869e-31,0.00915737,"Holz,Holz- u.Korkwaren (oh.Möbel),Flecht- u.Korbw.",3.22615e-34


In [23]:
sum(solution.value_added)

0.7952314870118232

In [7]:
BeyondHulten.cobb_douglas_costfun(ones(71),data.λ,data)

71×1 Matrix{Float64}:
 1.0
 1.0
 1.0
 1.0
 1.0
 1.0
 1.0
 1.0
 1.0
 1.0
 ⋮
 1.0
 1.0
 0.9999999999999999
 1.0
 1.0
 1.0
 1.0
 1.0
 0.9999999999999999

In [64]:
BeyondHulten.cobb_douglas_intermediary_demand(ones(71),data.λ,data) + data.consumption_share - data.λ

71×1 Matrix{Float64}:
 -0.0032089428680924445
 -0.0006171721909783655
 -8.04464753835775e-5
 -0.00044965024740240055
 -0.00016876821461393727
 -0.0016132171950116081
  0.10792192447740773
 -0.0018084003229694966
  0.0015772134626474837
 -0.0002363787636987108
  ⋮
 -0.002265094071221243
 -4.5594460607679965e-5
  0.013650625859659316
 -0.0057073033323794195
 -0.0021920328281819657
 -0.0023967958003346355
 -0.002973577442285139
 -0.0004268464195477234
 -0.0017276353695785153

In [54]:
(data.Ω' * (1 .- data.factor_share) .* data.λ) - sum(data.labor_share) * data.consumption_share)

Base.Meta.ParseError: ParseError:
# Error @ /home/franzs/Schreibtisch/BeyondHulten/CobbDouglas.ipynb:1:96
(data.Ω' * (1 .- data.factor_share) .* data.λ) - sum(data.labor_share) * data.consumption_share)
#                                                                                              ╙ ── extra tokens after end of expression

In [62]:
(data.Ω' * ((1 .- data.factor_share) .* data.λ)) + data.consumption_share - data.λ


71-element Vector{Float64}:
 -3.469446951953614e-18
  0.0
 -2.710505431213761e-20
  0.0
  5.421010862427522e-20
 -8.673617379884035e-19
  0.0
 -1.734723475976807e-18
  0.0
 -3.469446951953614e-18
  ⋮
  0.0
  6.938893903907228e-18
 -2.7755575615628914e-17
  0.0
  1.734723475976807e-18
  0.0
  0.0
 -2.168404344971009e-19
  3.469446951953614e-18